In [57]:
!pip install tensorflow
# https://github.com/zephyrproject-rtos/zephyr/blob/main/samples/modules/tflite-micro/hello_world/train/train_hello_world_model.ipynb
# https://colab.research.google.com/github/instafluff/TensorFlowLiteMicro_MNIST/blob/master/mnist_model_tensorflow_lite_micro.ipynb#scrollTo=F9sgeZzVtzzd
# https://www.tensorflow.org/datasets/keras_example

In [58]:
!unzip numbers-capture.zip -d numbers-capture

Archive:  numbers-capture.zip
replace numbers-capture/0/4.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [59]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input


MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model.keras'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [60]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# in numbers-capture directory there ten directories named 0, 1, 2, ... 9 with different number of photos inside

custom_mnist = dict()

for directory in os.listdir("/content/numbers-capture"):
  custom_mnist[directory] = []
  for file in os.listdir(f"/content/numbers-capture/{directory}"):
    custom_mnist[directory].append(f"/content/numbers-capture/{directory}/{file}")

x_train = []
y_train = []
x_test = []
y_test = []
test_size = 0.1

for digit, file_list in custom_mnist.items():
    # Determine how many images go to the test set
    num_test_images = int(len(file_list) * test_size)

    # Split the files into training and testing sets
    test_files = file_list[:num_test_images]
    train_files = file_list[num_test_images:]

    # Load test images and labels
    for file_path in test_files:
        img = Image.open(file_path).convert('L') # Convert to grayscale
        img = img.resize((28, 28)) # Resize to 28x28
        x_test.append(np.array(img))
        y_test.append(int(digit))

    # Load training images and labels
    for file_path in train_files:
        img = Image.open(file_path).convert('L') # Convert to grayscale
        img = img.resize((28, 28)) # Resize to 28x28
        x_train.append(np.array(img))
        y_train.append(int(digit))

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train[..., None]  # (28,28,1)
x_test  = x_test[..., None]

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [67]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16, 4, activation='relu', input_shape=(28,28,1)),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(32, 4, activation='relu'),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25), # Add dropout
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=['accuracy']
)
model.summary()

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)              │ (None, 25, 25, 16)     │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 12, 12, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 9, 9, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,626 (53.23 KB)

 Trainable params: 13,626 (53.23 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.fit(x_train, y_train, epochs=75, batch_size=128)
model.save(MODEL_TF)
scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", scores[0])
print("Test accuracy:", scores[1])

Epoch 1/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.0728 - loss: 2.3466
Epoch 2/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.1258 - loss: 2.2825
Epoch 3/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2453 - loss: 2.2321
Epoch 4/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2581 - loss: 2.1962
Epoch 5/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2889 - loss: 2.1575
Epoch 6/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3220 - loss: 2.1007
Epoch 7/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3583 - loss: 2.0497
Epoch 8/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3935 - loss: 2.0099
Epoch 9/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3891 - loss: 1.9586
Epoch 10/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4178 - loss: 1.8978
Epoch 11/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4421 - loss: 1.7855
Epoch 12/75
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4671 - loss: 1.7263
Epo

In [69]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

model_no_quant_tflite = converter.convert()
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_data_gen():
    for image in tf.data.Dataset.from_tensor_slices(x_train) \
                                  .batch(1).take(100):
        yield [image]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type  = tf.uint8   # albo tf.int8
converter.inference_output_type = tf.uint8   # albo tf.int8
tflite_quant_model = converter.convert()
open(MODEL_TFLITE,'wb').write(tflite_quant_model)


Saved artifact at '/tmp/tmpatcnjetk'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_144')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132802684251856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132802681587472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132802681586896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132802681588240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132802681585936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132802681589008: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved artifact at '/tmp/tmp2wnattu_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_144')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1328

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


19016

In [70]:
interpreter = tf.lite.Interpreter(model_path=MODEL_TFLITE)

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Adjust the model interpreter to take 10,000 inputs at once instead of just 1
interpreter.resize_tensor_input(input_details[0]["index"], (22, 28, 28, 1))
interpreter.resize_tensor_input(output_details[0]["index"], (22, 10))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

in_scale, in_zero_point = input_details['quantization']
x_test_quant = (x_test.astype(np.float32) / in_scale + in_zero_point).round().astype(input_details['dtype'])

interpreter.set_tensor(input_details['index'], x_test_quant)

interpreter.invoke()

output_quant = interpreter.get_tensor(output_details['index'])
out_scale, out_zero_point = output_details['quantization']
output_dequant = (output_quant.astype(np.float32) - out_zero_point) * out_scale

# 8. Compute accuracy
preds = np.argmax(output_dequant, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(preds == y_test_labels)
print("TFLite int8 Accuracy:", accuracy)

TFLite int8 Accuracy: 0.9090909090909091


In [71]:
size_tf = os.path.getsize(MODEL_TF)
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)


print("Model.tf: %d bytes" % size_tf)
print("Model_no_quant.tflite: %d bytes" % size_no_quant_tflite)
print("Model.tflite: %d bytes" % size_tflite)

Model.tf: 198197 bytes
Model_no_quant.tflite: 57552 bytes
Model.tflite: 19016 bytes


In [72]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

!cat {MODEL_TFLITE_MICRO}

unsigned char g_model[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x90, 0x00, 0x00, 0x00, 0x10, 0x01, 0x00, 0x00,
  0x78, 0x38, 0x00, 0x00, 0x88, 0x38, 0x00, 0x00, 0x84, 0x49, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x9e, 0xc6, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x64, 0xff, 0xff, 0xff,
  0x10, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x6f, 0x75, 0x74, 0x70, 0x75, 0x74, 0x5f, 0x30, 0x00, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x42, 0xc7, 0xff, 0xff,
  0x04, 0x